In [150]:
%reset
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import torch
import numpy as np
import defmod as dm

#import matplotlib.pyplot as plt
#torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [151]:
dim = 2

sigma1 = 10
nb_pts1 = 5
manifold1 = dm.manifold.Landmarks(dim, nb_pts1)
gd1 = torch.rand(nb_pts1, dim)
manifold1.fill_gd(gd1.view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 10
nb_pts2 = 7
manifold2 = dm.manifold.Landmarks(dim, nb_pts2)
gd2 = torch.rand(nb_pts2, dim)
manifold2.fill_gd(gd2.view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

module_list = [trans1, trans2]
nb_pts = nb_pts1 + nb_pts2

In [152]:
sigma3 = 0.5
background = dm.deformationmodules.Background(module_list, sigma3)

In [153]:
cotan1 = torch.rand(gd1.shape)
trans1.manifold.fill_cotan(cotan1.view(-1))
trans1.compute_geodesic_control(manifold1)
trans1.controls

tensor([ -106.8655,   -77.3356, -2649.3103, -1919.0532,  1154.8353,   836.5274,
          820.7694,   594.3050,   784.2070,   568.1910])

In [154]:
cotan2 = torch.rand(gd2.shape)
trans2.manifold.fill_cotan(cotan2.view(-1))
trans2.compute_geodesic_control(manifold2)
trans2.controls

tensor([ 509.1888,  467.1892, -300.5175, -270.0637,  732.9759,  670.6016,
        -874.8680, -802.8270,  419.4366,  383.3792,  354.8319,  325.8596,
        -834.9889, -768.5864])

In [155]:
points = torch.rand(10,dim)
background(points)

tensor([[5.1253, 4.1117],
        [4.0947, 3.5938],
        [5.4377, 4.6252],
        [4.7057, 3.9960],
        [3.9165, 3.0639],
        [4.4556, 3.7093],
        [3.2463, 2.4978],
        [5.1433, 4.2923],
        [3.8632, 3.0356],
        [5.0834, 4.3522]])

In [156]:
manifold = dm.manifold.CompoundManifold([manifold1, manifold2])
TransBackground = dm.deformationmodules.CompoundModule(
    [dm.deformationmodules.Translations(manifold1,sigma3),
     dm.deformationmodules.Translations(manifold2,sigma3)])
TransBackground.compute_geodesic_control(manifold)
TransBackground.controls

[tensor([ 0.3267,  0.2116, -6.0122, -5.4001,  3.6470,  3.4244,  4.4150,  3.4963,
          6.0845,  5.3003]),
 tensor([ 3.0810,  2.2758, -1.3092, -1.2129,  3.3229,  2.7206, -0.6586, -0.2344,
          2.7204,  2.5766,  3.0013,  2.6456, -0.0591, -0.3488])]

In [157]:
background.compute_geodesic_control(manifold)
background.controls

[tensor([0.0960, 0.1609, 0.5366, 0.2947, 0.4861, 0.4987, 0.9331, 0.2690, 0.5265,
         0.6451]),
 tensor([0.1169, 0.2260, 0.7068, 0.0550, 0.6344, 0.7703, 0.7581, 0.4552, 0.6165,
         0.7063, 0.5587, 0.6089, 0.6303, 0.8636])]

In [172]:
background.cost()

tensor(13.5676)

In [175]:
TransBackground.manifold.fill_gd(background.manifold.gd)
TransBackground.fill_controls(background.controls)
TransBackground.cost()

tensor(13.5676)

In [171]:
background.dim


2